In [ ]:
import os
import sys

PROJECT_ROOT = '/app/'

if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

In [ ]:
# 导入可视化模块（字体配置已自动集成）
from backtesting.visualizer import get_font_info

print("=== 检查字体配置 ===")
# 字体配置已自动在visualizer初始化时完成
font_info = get_font_info()
print(f"当前字体: {font_info['current_font']}")
print(f"中文支持: {'是' if font_info['chinese_support'] else '否'}")
print("字体配置完成，准备开始分析")

In [ ]:
# 股票回测实现
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 重新导入回测框架以获取最新修改
import importlib
import backtesting.backtest
importlib.reload(backtesting.backtest)
from backtesting.backtest import SimpleBacktest

print("=== 招商银行股票回测分析 ===")

# 1. 读取CSV数据
csv_file_path = '/app/data/stockdata/sh.600036.招商银行.csv'
print(f"\n正在读取数据文件: {csv_file_path}")

try:
    # 读取CSV数据
    data = pd.read_csv(csv_file_path)
    print(f"成功读取数据，共 {len(data)} 条记录")
    print(f"数据时间范围: {data['date'].min()} 到 {data['date'].max()}")
    
    # 数据预处理
    data['date'] = pd.to_datetime(data['date'])
    data = data.sort_values('date').reset_index(drop=True)
    
    # 计算日收益率
    data['pct_change'] = data['close'].pct_change()
    
    # 计算移动平均线
    data['ma5'] = data['close'].rolling(window=5).mean()
    data['ma20'] = data['close'].rolling(window=20).mean()
    
    # 显示数据基本信息
    print("\n数据前5行:")
    print(data[['date', 'open', 'high', 'low', 'close', 'volume', 'pct_change']].tail())
    
except Exception as e:
    print(f"读取数据失败: {e}")
    data = None

In [ ]:
# 2. 定义回测策略
def ma_crossover_strategy(index: int, row: pd.Series, backtest: SimpleBacktest, data: pd.DataFrame) -> str:
    """
    均线交叉策略：
    - 当5日均线上穿20日均线时买入
    - 当5日均线下穿20日均线时卖出
    """
    # 需要足够的历史数据计算均线
    if index < 20 or pd.isna(row['ma5']) or pd.isna(row['ma20']):
        return 'hold'
    
    # 获取前一天的均线数据
    if index > 0:
        prev_ma5 = data.iloc[index-1]['ma5'] if index-1 < len(data) else None
        prev_ma20 = data.iloc[index-1]['ma20'] if index-1 < len(data) else None
        
        if prev_ma5 is not None and prev_ma20 is not None:
            # 金叉：5日线上穿20日线，买入
            if prev_ma5 <= prev_ma20 and row['ma5'] > row['ma20'] and backtest.position == 0:
                return 'buy'
            # 死叉：5日线下穿20日线，卖出
            elif prev_ma5 >= prev_ma20 and row['ma5'] < row['ma20'] and backtest.position > 0:
                return 'sell'
    
    return 'hold'

def momentum_strategy(index: int, row: pd.Series, backtest: SimpleBacktest, data: pd.DataFrame) -> str:
    """
    动量策略：
    - 当日涨幅超过3%时买入
    - 当日跌幅超过2%时卖出
    """
    if pd.isna(row['pct_change']):
        return 'hold'
    
    # 涨幅超过3%买入
    if row['pct_change'] > 0.03 and backtest.position == 0:
        return 'buy'
    # 跌幅超过2%卖出
    elif row['pct_change'] < -0.02 and backtest.position > 0:
        return 'sell'
    
    return 'hold'

def rsi_strategy(index: int, row: pd.Series, backtest: SimpleBacktest, data: pd.DataFrame) -> str:
    """
    简化的RSI策略：
    - 用价格在最近 14 天区间中的相对位置，低位买入，高位卖出，模拟 RSI 的超买超卖逻辑。
    """
    if index < 14:  # 需要足够数据计算RSI
        return 'hold'
    
    # 计算14日内的价格相对位置
    recent_data = data.iloc[max(0, index-13):index+1]
    min_price = recent_data['close'].min()
    max_price = recent_data['close'].max()

    if max_price > min_price:
        price_position = (row['close'] - min_price) / (max_price - min_price)

        # 价格在低位时买入
        if price_position < 0.3 and backtest.position == 0:
            return 'buy'
        # 价格在高位时卖出
        elif price_position > 0.7 and backtest.position > 0:
            return 'sell'
    
    return 'hold'

print("策略函数定义完成！")

In [ ]:
def breakout_strategy(index: int, row: pd.Series, backtest: SimpleBacktest, data: pd.DataFrame):
    """
    突破策略：
    - 当前价格位于五日均线下方且当天的收盘价低于前一日最低价时全仓卖出
    - 当前价格位于五日均线上方且当天的收盘价高于前一日最高价时买入1/3仓位
    """
    if index < 1 or pd.isna(row['ma5']):
        return 'hold'
    
    prev_low = data.iloc[index-1]['low']
    prev_high = data.iloc[index-1]['high']
    
    # 当前价格位于五日均线下方且当天的收盘价低于前一日最低价时全仓卖出
    if row['close'] < row['ma5'] and row['close'] < prev_low and backtest.position > 0:
        return {'action': 'sell', 'ratio': 1.0}  # 全仓卖出
    
    # 当前价格位于五日均线上方,五日均线高于二十日均线，且当天的收盘价高于前一日最高价时买入
    if row['close'] > row['ma5'] and row['ma5'] > row['ma20'] and row['close'] > prev_high and backtest.cash > 0:
    #if row['close'] > row['ma5'] and row['close'] > prev_high and backtest.cash > 0:
        return {'action': 'buy', 'ratio': 1.0}
    
    return 'hold'

In [ ]:
last_trade_price = 10000  # 初始值设为一个较大值，确保首次买入
def price_threshold_strategy(index: int, row: pd.Series, backtest: SimpleBacktest, data: pd.DataFrame):
    """
    价格阈值策略：
    - 当价格高于上次交易价格的5%时卖出
    - 当价格低于上次交易价格的5%时买入
    """
    global last_trade_price
    
    if backtest.position > 0:  # 如果有持仓，检查是否需要卖出
        if row['close'] >= last_trade_price * 1.05:
            last_trade_price = row['close']  # 更新最后交易价格
            return 'sell'
    
    if backtest.cash > 0:  # 如果有现金，检查是否需要买入
        if row['close'] <= last_trade_price * 0.90:
            last_trade_price = row['close']  # 更新最后交易价格
            return 'buy'
    
    return 'hold'

In [ ]:
# 3. 执行回测
if data is not None:
    print("\n=== 开始回测 ===")
    
    # 选择最近5年的数据进行回测（如果数据足够）
    recent_data = data.tail(1250).copy()  # 大约5年交易日
    #recent_data = data.tail(250).copy()  # 大约5年交易日
    recent_data = recent_data.reset_index(drop=True)
    
    print(f"回测数据范围: {recent_data['date'].min().strftime('%Y-%m-%d')} 到 {recent_data['date'].max().strftime('%Y-%m-%d')}")
    print(f"回测数据量: {len(recent_data)} 条")
    
    # 策略列表
    strategies = {
        '均线交叉策略': ma_crossover_strategy,
        #'动量策略': momentum_strategy,
        #'RSI策略': rsi_strategy
        "突破策略": breakout_strategy,
        "价格阈值策略": price_threshold_strategy
    }
    
    # 存储所有策略的回测结果
    all_results = {}
    
    for strategy_name, strategy_func in strategies.items():
        print(f"\n--- 执行 {strategy_name} ---")
        
        # 创建回测实例
        bt = SimpleBacktest(initial_cash=100000)
        
        # 运行回测
        try:
            results = bt.run_backtest(recent_data, strategy_func)
            all_results[strategy_name] = results
            
            # 打印策略结果摘要
            bt.print_summary(results)
            
        except Exception as e:
            print(f"{strategy_name} 回测失败: {e}")
    
    print("\n=== 回测完成 ===")
else:
    print("数据加载失败，无法进行回测")

In [ ]:
# 5. 可视化分析
from backtesting.visualizer import plot_backtest_results

if all_results:
    print("\n=== 绘制回测结果图表 ===")
    
    # 使用新的可视化模块绘制对比图表
    plot_backtest_results(all_results, recent_data)
    
    print("✅ 图表绘制完成！")
else:
    print("没有回测结果可绘制")

In [ ]:
# 6. 详细交易记录分析（数据统计）
if all_results:
    print("\n=== 交易记录详细分析 ===")
    
    # 选择最佳策略进行详细分析
    best_strategy_name = max(all_results.items(), key=lambda x: x[1]['total_return'])[0]
    best_strategy_name = '突破策略'
    best_results = all_results[best_strategy_name]
    
    print(f"\n📊 详细分析最佳策略: {best_strategy_name}")
    
    if not best_results['trades'].empty:
        trades_df = best_results['trades']
        
        # 交易统计
        buy_trades = trades_df[trades_df['action'] == 'buy']
        sell_trades = trades_df[trades_df['action'] == 'sell']
        
        print(f"\n交易统计:")
        print(f"  总买入次数: {len(buy_trades)}")
        print(f"  总卖出次数: {len(sell_trades)}")
        
        if len(buy_trades) > 0 and len(sell_trades) > 0:
            # 计算每笔交易的盈亏
            trade_profits = []
            for i in range(min(len(buy_trades), len(sell_trades))):
                buy_price = buy_trades.iloc[i]['price']
                sell_price = sell_trades.iloc[i]['price']
                profit_rate = (sell_price - buy_price) / buy_price
                trade_profits.append(profit_rate)
            
            if trade_profits:
                print(f"  平均每笔交易收益率: {np.mean(trade_profits):.2%}")
                print(f"  最大单笔盈利: {max(trade_profits):.2%}")
                print(f"  最大单笔亏损: {min(trade_profits):.2%}")
                print(f"  盈利交易比例: {sum(1 for p in trade_profits if p > 0) / len(trade_profits):.2%}")
        
        # 显示最近几笔交易
        print(f"\n最近10笔交易记录:")
        recent_trades = trades_df.tail(10)[['date', 'action', 'price', 'volume']]
        if not recent_trades.empty:
            recent_trades['date'] = pd.to_datetime(recent_trades['date']).dt.strftime('%Y-%m-%d')
            print(recent_trades.to_string(index=False))
    else:
        print("该策略没有产生交易记录")
    
    # 月度收益分析
    history_df = best_results['history']
    if not history_df.empty:
        history_df['date'] = pd.to_datetime(history_df['date'])
        history_df['month'] = history_df['date'].dt.to_period('M')
        
        # 计算月度收益
        monthly_returns = history_df.groupby('month')['total_value'].agg(['first', 'last'])
        monthly_returns['return'] = (monthly_returns['last'] - monthly_returns['first']) / monthly_returns['first']
        
        print(f"\n📅 月度收益分析 (最近12个月):")
        recent_monthly = monthly_returns.tail(12)
        for month, row in recent_monthly.iterrows():
            print(f"  {month}: {row['return']:+.2%}")
        
        # 月度收益统计
        positive_months = (monthly_returns['return'] > 0).sum()
        total_months = len(monthly_returns)
        print(f"\n月度胜率: {positive_months}/{total_months} ({positive_months/total_months:.2%})")

print("\n📈 数据分析完成！详细图表请查看上方可视化结果。")

In [ ]:
# 6.1 最佳策略详细分析（使用可视化模块）
from backtesting.visualizer import plot_single_strategy_analysis, plot_trade_details, plot_monthly_performance

if all_results:
    print("\n=== 最佳策略详细可视化分析 ===")
    
    # 选择最佳策略
    #best_strategy_name = max(all_results.items(), key=lambda x: x[1]['total_return'])[0]
    best_strategy_name = '突破策略'
    best_results = all_results[best_strategy_name]
    
    print(f"正在分析最佳策略: {best_strategy_name}")
    
    # 1. 策略整体分析
    print("\n1. 策略整体表现分析:")
    plot_single_strategy_analysis(best_results, best_strategy_name, recent_data)
    
    # 2. 交易详细分析
    print("\n2. 交易详细分析:")
    plot_trade_details(best_results, best_strategy_name)
    
    # 3. 月度表现分析
    print("\n3. 月度表现分析:")
    plot_monthly_performance(best_results, best_strategy_name)
    
    print(f"\n✅ {best_strategy_name} 详细分析完成！")
else:
    print("没有回测结果可分析")